# Data Extraction
This notebook is a sandbox for extracting all of the post-cohort IEQ data from the csvs and converting to a more useful object, i.e a Pandas DataFrame.

## Imports

In [1]:
import pandas as pd
import os
import sys

## Read in the Data

In [2]:
def get_file_list(data_directory):
    '''
    Return a list of all the csvs within a directory
    '''
    file_list = []
    for root, dirs, files in os.walk(data_directory):
        for name in files:
            if name.endswith('.csv'):
                file_name = os.path.join(root, name)
                file_list.append(file_name)
    
    return file_list

In [3]:
def parse_file_name(file_name):
    '''
    Given a csv's file name, extract the building, floor, room and room_type
    '''
    file = file_name.split("/")[-1]
    date = file.split("_")[0]
    location_info = file.replace('.csv','').replace(date,'').replace('_if_us_dc_','')
    building, floor, room_and_type = location_info.split("_")
    if '-' in room_and_type:
        room_and_type_split = room_and_type.split("-")
        room = room_and_type_split[0]
        room_type = room_and_type_split[1]
    else:
        room = room_and_type
        room_type = None
    
    return building, floor, room, room_type



In [4]:
def csv_to_df(file_name):
    '''
    Given the path to a csv, extract location data from the file name 
    and return that along with the data as a pandas DataFrame
    '''
    df = pd.read_csv(file_name, 
                     parse_dates = [0], 
                     dtype = 'float')
    cols = df.columns
    df = df.rename(mapper = {'Unnamed: 0':'Date Time', 'datetime':'Date Time'},
                   axis = 'columns')
    building, floor, room, room_type = parse_file_name(file_name)
    df['building'] = building
    df['floor'] = floor
    df['room'] = room
    df['room_type'] = room_type
    
    return df
    

In [9]:
def make_final_df():
    file_list = get_file_list('data/post-cohort')
    dfs = []
    for f in file_list:
        df = csv_to_df(f)
        dfs.append(df)
    final_df = pd.concat(dfs, ignore_index=True, sort=True)
    
    return final_df

In [10]:
final_df = make_final_df()

In [11]:
final_df.head()

,CO2__ppm,CO__ppm,Date Time,Humidity__pct,PN1Plus__counts_per_L,Pressure__mbar,Temperature__deg_C,building,floor,humidity_2_rel_true_pct_avg,light_lux,room,room_type,sound_dba_avg,temperature_2_celsius_avg,vibration_ms2_avg
0,533.073,0.827,2017-02-24 00:00:00,47.583,130.622,1008.922,22.788,1800f,2,49.143,3.679,2462,None,46.562,22.755,0.102
1,530.667,0.831,2017-02-24 00:01:00,47.618,NaN,1008.913,22.786,1800f,2,49.162,3.756,2462,None,46.587,22.750,0.103
2,522.754,0.830,2017-02-24 00:02:00,47.626,NaN,1008.925,22.788,1800f,2,49.185,4.072,2462,None,46.587,22.750,0.104
3,527.022,0.826,2017-02-24 00:03:00,47.656,NaN,1008.947,22.787,1800f,2,49.197,4.065,2462,None,46.570,22.747,0.104
4,529.830,0.827,2017-02-24 00:04:00,47.633,NaN,1008.968,22.781,1800f,2,49.193,4.062,2462,None,46.571,22.741,0.103


# Transform the Data

In [12]:
def clean_df(long_df):
    '''
    Take long dataframe and split 'Measure' column into two new columns, capturing
    'Modality' and 'Unit'. Drop 'Measure' column.
    '''
    temp = long_df['Measure'].astype('str').str.split('_', n=1, expand=True)
    long_df['Modality'] = temp[0]
    long_df['Unit'] = temp[1].str.replace("_", " ")
    clean_df = long_df.drop(['Measure'], axis=1)
    
    return clean_df    

def reshape_df(final_df):
    '''
    Take wide dataframe and transform so that id_cols are constant for all measures
    and dataframe columns not in id_cols (value_cols) are tranformed into two columns:
    'Measure' inlcudes previous column name and 'Value' includes previous columns value
    with updated long dataframe returned.
    '''
    if 'Unnamed: 0' in final_df.columns:
        final_df = final_df.drop('Unnamed: 0', axis = 1)
    id_cols = {'Date Time', 'building', 'floor', 'room','room_type'}
    value_cols = set(final_df.columns) - id_cols
    long_df = pd.melt(final_df, 
                      id_vars= list(id_cols), 
                      value_vars= list(value_cols),
                      var_name='Measure', 
                      value_name='Value')
    
    df = clean_df(long_df)
    df = df.dropna(subset = ['Value'])
    
    return df

In [13]:
df = reshape_df(final_df)

In [14]:
df.shape

(144169327, 8)

In [15]:
df.head()

,building,floor,Date Time,room,room_type,Value,Modality,Unit
0,1800f,2,2017-02-24 00:00:00,2462,None,22.755,temperature,2 celsius avg
1,1800f,2,2017-02-24 00:01:00,2462,None,22.750,temperature,2 celsius avg
2,1800f,2,2017-02-24 00:02:00,2462,None,22.750,temperature,2 celsius avg
3,1800f,2,2017-02-24 00:03:00,2462,None,22.747,temperature,2 celsius avg
4,1800f,2,2017-02-24 00:04:00,2462,None,22.741,temperature,2 celsius avg


# Write each modality to csv

In [17]:
for modality in df['Modality'].unique():
    modality_df = df[df['Modality'] == modality]
    out_path = os.path.join('data','post-cohort', 'parsed-data')
    if not os.path.exists(out_path):
        os.mkdir(out_path)
    file_out_path = os.path.join(out_path, f'{modality}-post-cohort.csv')
    modality_df.to_csv(file_out_path, index = False)